In [1]:
import os,sys
sys.path.append('../')
from utils.io_utils import *
from utils.ftx_utils import *
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [2]:
#i=0 # uncomment to reset

curdir = os.path.join(os.sep,'tmp','tradeexecutor')
logdir = os.path.join(os.sep,'tmp','tradeexecutor','archive')
if i==0:
    already_opened = []
    risk= pd.DataFrame()
    i += 1
for filename in [os.path.join(os.sep,'tmp','tradeexecutor','latest_risk_reconciliations')] + os.listdir(logdir):
    if 'risk_reconciliations.json' in filename and filename not in already_opened:
        with open(os.path.join(os.sep,logdir,filename),'r') as fp:
            new_risk = pd.DataFrame(json.load(fp))
            new_risk['delta_timestamp'] = (new_risk['delta_timestamp']/1000).apply(datetime.utcfromtimestamp)
        risk = pd.concat([risk ,new_risk])
        already_opened += [filename]

In [3]:
risk['coin'] = risk['symbol'].apply(lambda f: f.replace('/USD','').replace(':USD',''))

In [4]:
pv = pd.DataFrame(risk.groupby(by='delta_timestamp')['pv'].mean()).iplot()

In [5]:
margins = pd.DataFrame(risk.groupby(by='delta_timestamp')[['estimated_IM','actual_IM']].mean()).iplot()

In [6]:
cash_symbols = [symbol for symbol in risk['symbol'].unique() if ':' not in symbol]
carry_delta = pd.DataFrame(risk[risk['symbol'].isin(cash_symbols)].groupby(by='delta_timestamp')['delta'].sum()).iplot()

In [11]:
netDelta = risk.pivot_table(index='delta_timestamp',columns='coin',values='netDelta',aggfunc=np.mean).iplot()

In [ ]:
fig = make_subplots(rows=len(parsed), cols=1,
                    subplot_titles=list(parsed.keys()),
                    shared_xaxes=True,
                    vertical_spacing=0.02)

for i,(name,data) in enumerate(parsed.items()):
    for mode in data:
        fig.append_trace(go.Scatter(x=data[mode].index,y=data[mode].values,name=mode),
            row=i+1, col=1)

fig.update_layout(height=1000, width=1000, title_text="live metrics")
fig.show()